<a href="https://colab.research.google.com/github/mihirkumar28/HPC-SPPU-BE/blob/main/HPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-cg-toolkit' for glob 'nvidia*'
N

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2020-09-18 05:43:44--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?IVg_UeR5e3vVOmG-nWIwtGsCV7mJjiBUu_tHQjFSh8NwCNIm2LBW-apQX8BEG-yx1zyB7n-XIslUc03Nl2iUcqqBwBBVlLt8TkYNJ6l-lndaK36b3x0Cf2zAfDWKkrTjxDudHCDI1wf7UkLHGgyrBJrlUqJ39JQOUMd90GC_ePlM-_XVT2-IyqW5mEPb3HSvz_4b90klzIToj8AtaWk [following]
--2020-09-18 05:43:44--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?IVg_UeR5e3vVOmG-nWIwtGsCV7mJjiBUu_tHQjFSh8NwCNIm2LBW-apQX8BEG-yx1zyB7n-XIslUc03Nl2iUcqqBwBBVlLt8TkYNJ6l-lndaK36b3x0Cf2zAfDWK

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-p0pa47wr
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-p0pa47wr
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=8e49ba60b4f6cb038aa881491772fa12fc82859539c33ac77321383e328dd4d6
  Stored in directory: /tmp/pip-ephem-wheel-cache-f31z_sep/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
 
__global__ void add(int *a, int *b, int *c) {
 
 
 
  c[blockIdx.x]=a[blockIdx.x]+b[blockIdx.x];
}
 
 
int main() {
 
int *d_a, *d_b, *d_c;
 
int h_c[6];
int h_a[6] = {1,2,3,4,5,6};
int h_b[6] = {7,8,9,10,11,12};
 
 
int size = sizeof(int);
cudaMalloc((void **)&d_a, 6*size);
cudaMalloc((void **)&d_b, 6*size);
cudaMalloc((void **)&d_c, 6*size);
 
cudaMemcpy(d_a, &h_a, 6*size, cudaMemcpyHostToDevice);
cudaMemcpy(d_b, &h_b, 6*size, cudaMemcpyHostToDevice);
 
clock_t time;
time=clock();
add<<<6,1>>>(d_a, d_b, d_c);
 
 
cudaError err = cudaMemcpy(&h_c, d_c, 6*size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
 
for(int i=0;i<6;i++){
    printf("Sum of %d & %d is %d\n",h_a[i],h_b[i],h_c[i]);
}
 
time=clock()-time;
double time_taken = ((double)time)/CLOCKS_PER_SEC;
printf("Time taken is %f\n",time_taken);
 
 
 
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
 
 
 
return 0;
}

Sum of 1 & 7 is 8
Sum of 2 & 8 is 10
Sum of 3 & 9 is 12
Sum of 4 & 10 is 14
Sum of 5 & 11 is 16
Sum of 6 & 12 is 18
Time taken is 0.000039



In [ ]:
%%cu

        
#include<iostream>
#include<cstdio>
#include<cstdlib>
#include<cuda_runtime.h>
using namespace std;


__global__ void minimum(int *input)
{
	int tid=threadIdx.x;
	auto step_size=1;
  int number_of_threads=blockDim.x;
  
  while(number_of_threads>0)
  {
      if(tid<number_of_threads)
      {
          int first=tid*step_size*2;
          int second=first+step_size;
          if(input[second]<input[first])
            input[first]=input[second];
      }
      step_size=step_size*2;
      number_of_threads/=2;
  }

}

__global__ void max(int *input)
{
   int tid=threadIdx.x;
   auto step_size=1;
   int number_of_threads=blockDim.x;
   
   while(number_of_threads>0)
   {
       if(tid<number_of_threads)
       {
           int first=tid*step_size*2;
           int second=first+step_size;
           if(input[second]>input[first])
            input[first]=input[second];
       }
       step_size*=2;
       number_of_threads/=2;
   }
}


int main()
{

	cout<<"Enter the no of elements"<<endl;
	int n;
	n=100;
  srand(n);
	int *input=new int[n];

	for(int i=0;i<n;i++)
	{
		input[i]=rand();
    cout<<input[i]<<" ";
	}

	int size=n*sizeof(int); 
	int *mini,result_min;
	
  
	cudaMalloc(&mini,size);
	cudaMemcpy(mini,input,size,cudaMemcpyHostToDevice);

  minimum<<<1,n/2>>>(mini);

	cudaMemcpy(&result_min,mini,sizeof(int),cudaMemcpyDeviceToHost);

	cout<<"\nThe minimum element is "<<result_min<<endl;
   
  int *input_max,maxValue;
  cudaMalloc(&input_max,size);
	cudaMemcpy(input_max,input,size,cudaMemcpyHostToDevice);

  max<<<1,n/2>>>(input_max);

	cudaMemcpy(&maxValue,input_max,sizeof(int),cudaMemcpyDeviceToHost);

	cout<<"The maximum element is \n "<<maxValue<<endl;


   cudaFree(mini);
   cudaFree(input_max);
    
    
 

return 0;

}

Enter the no of elements
677741240 611911301 516687479 1039653884 807009856 115325623 1224653905 2083069270 1106860981 922406371 876420180 322411828 1259609597 1356425747 1318253490 883344301 229978180 1871801538 567764310 1334753943 1440409294 768869741 447440212 1147132415 825732306 1813670413 1897413451 991129617 1396963907 1658754524 1065898046 2074705148 123182177 1582585525 966875384 930192033 1697911148 44045642 865777656 657288481 966452013 1742197836 979700309 78577962 951139936 150470151 961922264 1181118116 2022271690 1529686574 368388411 1315197336 151072667 815828623 314846104 976804973 482015388 64775907 1967934590 1878979296 1723530431 886348988 1806200796 1846712609 321450865 625592532 629420994 2019362013 669638174 1495198650 529166846 1636090187 1089912839 1508867155 1714668150 2041052775 1659337307 529106766 1074687243 1534125349 2058793340 1443075654 701839037 62382359 111420630 1016685141 1039187332 593436018 1081461048 859638275 324931666 657507832 1745987263 2131

In [ ]:
%%cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <iostream>
#include <math.h>
#include <numeric>
using namespace std;

__global__ void sum(int* input)
{
	const int tid = threadIdx.x;

	auto step_size = 1;
	int number_of_threads = blockDim.x;

	while (number_of_threads > 0)
	{
		if (tid < number_of_threads)
		{
			const auto fst = tid * step_size * 2;
			const auto snd = fst + step_size;
			input[fst] += input[snd];
		}

		step_size <<= 1; 
		number_of_threads >>= 1;
	}
}


int main()
{
	int n;
  n=100;
  srand(n);
  int *arr=new int[n];
   
    for(int i=0;i<n;i++)
    {
      arr[i]=rand()%20000;
      cout<<arr[i]<<" ";
    }

	int* d;
  int size=n*sizeof(int);
	
	cudaMalloc(&d, size);
	cudaMemcpy(d, arr, size, cudaMemcpyHostToDevice);

	sum <<<1, n / 2 >>>(d);

	int result;
	cudaMemcpy(&result, d, sizeof(int), cudaMemcpyDeviceToHost);
	cout << "\nSum is " << result << endl;



	
  cout << "Average is " << result/n << endl;
	
	getchar();

	cudaFree(d);
	delete[] arr;

	return 0;
}

1240 11301 7479 13884 9856 5623 13905 9270 981 6371 180 11828 9597 5747 13490 4301 18180 1538 4310 13943 9294 9741 212 12415 12306 10413 13451 9617 3907 14524 18046 5148 2177 5525 15384 12033 11148 5642 17656 8481 12013 17836 309 17962 19936 10151 2264 18116 11690 6574 8411 17336 12667 8623 6104 4973 15388 15907 14590 19296 10431 8988 796 12609 10865 12532 994 2013 18174 18650 6846 10187 12839 7155 8150 12775 17307 6766 7243 5349 13340 15654 19037 2359 630 5141 7332 16018 1048 18275 11666 7832 7263 12462 16793 18129 1347 17787 16494 19521 
Sum is 633114
Average is 6331



In [ ]:
%%writefile test.cpp
#include<omp.h>
#include<iostream>
using namespace std;

int main() {
    int n = 100;
    srand(n);
    int *arr=new int[n];
    
    
    omp_set_num_threads(2);
    
    for(int i=0; i<n; i++) {
        arr[i] = rand()% 1000;
    }
    cout<<"Unsorted List:\n";
    for(int f=0; f<n; f++)
      cout<<"  "<<arr[f];
    
    int start=0,k=0,l=0;
    double start_time, end_time;
    
    start_time = omp_get_wtime();
    for(k=0; k<n-1; k++) {
        start = k%2;
        #pragma omp parallel for
        for(l=start; l<n-1; l++) {
            if(arr[l] > arr[l+1])
              swap(arr[l], arr[l+1]);
        }
    }
    end_time = omp_get_wtime();
    cout<<"Sorted array: "<<endl;
    for(int z=0; z<n; z++)
      cout<<"  "<<arr[z];
    cout<<endl;
    
    cout<<"Parallel execution time = "<<(end_time-start_time)<<endl;
    
    return 0;
    
}

Overwriting test.cpp


In [ ]:
!g++ -fopenmp test.cpp 

In [ ]:
!./a.out

Unsorted List:
  240  301  479  884  856  623  905  270  981  371  180  828  597  747  490  301  180  538  310  943  294  741  212  415  306  413  451  617  907  524  46  148  177  525  384  33  148  642  656  481  13  836  309  962  936  151  264  116  690  574  411  336  667  623  104  973  388  907  590  296  431  988  796  609  865  532  994  13  174  650  846  187  839  155  150  775  307  766  243  349  340  654  37  359  630  141  332  18  48  275  666  832  263  462  793  129  347  787  494  521Sorted array: 
  13  13  18  33  37  46  48  104  116  129  141  148  148  150  151  155  174  177  180  180  187  212  240  243  263  264  270  275  294  296  301  301  306  307  309  310  332  336  340  347  349  359  371  384  388  411  413  415  431  451  462  479  481  490  494  521  524  525  532  538  574  590  597  609  617  623  623  630  642  650  654  656  666  667  690  741  747  766  775  787  793  796  828  832  836  839  846  856  865  884  905  907  907  936  943  962  97

In [ ]:
%%writefile mihir.cpp
#include<iostream>
#include<cstdlib>
#include<omp.h>
using namespace std;


void merge(int *arr,int l,int mid,int r)
{
	int n1 = mid-l+1;
	int n2 = r-mid;
	
	int i,j,k;
	
	int *a = new int[n1];
	int *b = new int[n2];
	
	for(i=0;i<n1;i++)
		a[i] = arr[i+l];
		
	for(i=0;i<n2;i++)
		b[i] = arr[i+mid+1];
		
	k = l;
	i = 0;
	j = 0;
	
	while(i<n1 && j<n2)
	{
		if(a[i]<b[j])
		{
			arr[k] = a[i];
			k++;
			i++;
		}
		
		else
		{
			arr[k] = b[j];
			k++;
			j++;
		}
	}
	
	while(i<n1)
	{
		arr[k] = a[i];
		k++;
		i++;
	}
	
	while(j<n2)
	{
		arr[k] = b[j];
		k++;
		j++;
	}
}

void mergesort(int *arr,int l, int r)
{
	if(l<r)
	{
		int mid = (l+r)/2;
		
		mergesort(arr,l,mid);
		mergesort(arr,mid+1,r);
		
		merge(arr,l,mid,r);
	}
}

void parallel_mergesort(int *arr,int l,int r)
{
	if(l<r)
	{
		int mid = (l+r)/2;
	
		#pragma omp parallel sections num_threads(2)
		{
			#pragma omp section
			{
				mergesort(arr,l,mid);
			}
			
			#pragma omp section
			{
				mergesort(arr,mid+1,r);
			}
		}
		
		merge(arr,l,mid,r);
	}
}

int main()
{
	int n = 100;
	
	int *arr = new int[n];
	
	srand(100);
	cout<<"\nUnsorted Array:\n";
	for(int i = 0; i < n; i++)
	{
		arr[i] = rand()%1000;
		cout<<arr[i]<<" ";
	}

	double start = omp_get_wtime();
	
	mergesort(arr,0,n-1);
	
	double end = omp_get_wtime();
	
	cout << "\nSequential Execution: "<<(end - start) << "\nSorted Array:\n";
  
  for(int i=0;i<n;i++)
      cout<<arr[i]<<" ";
 cout<<endl<<endl;
	
	double start1 = omp_get_wtime();
	

	parallel_mergesort(arr,0,n-1);
	
	double end1 = omp_get_wtime();	

	cout << "\nParallel Execution: " <<(end1 - start1) << "\nSoretd Array\n";

  
  for(int i=0;i<n;i++)
      cout<<arr[i]<<" ";
		
	
	return(0);
}


In [ ]:
!g++ -fopenmp mihir.cpp 


In [ ]:
!./a.out


Unsorted Array:
240 301 479 884 856 623 905 270 981 371 180 828 597 747 490 301 180 538 310 943 294 741 212 415 306 413 451 617 907 524 46 148 177 525 384 33 148 642 656 481 13 836 309 962 936 151 264 116 690 574 411 336 667 623 104 973 388 907 590 296 431 988 796 609 865 532 994 13 174 650 846 187 839 155 150 775 307 766 243 349 340 654 37 359 630 141 332 18 48 275 666 832 263 462 793 129 347 787 494 521 
Sequential Execution: 3.4027e-05
Sorted Array:
13 13 18 33 37 46 48 104 116 129 141 148 148 150 151 155 174 177 180 180 187 212 240 243 263 264 270 275 294 296 301 301 306 307 309 310 332 336 340 347 349 359 371 384 388 411 413 415 431 451 462 479 481 490 494 521 524 525 532 538 574 590 597 609 617 623 623 630 642 650 654 656 666 667 690 741 747 766 775 787 793 796 828 832 836 839 846 856 865 884 905 907 907 936 943 962 973 981 988 994 


Parallel Execution: 0.000174603
Soretd Array
13 13 18 33 37 46 48 104 116 129 141 148 148 150 151 155 174 177 180 180 187 212 240 243 263 264 270 

In [ ]:
%%cu

#include <iostream>
#include <cstdlib>
#include <chrono>

using namespace std;
using namespace std::chrono;

#define M 1024*1024
#define N 1024

#define BLOCKSIZE 32


__global__ void mat_vec_mul_kernel(float *a, float *b, float *c) {

    int tidx = blockIdx.x * blockDim.x + threadIdx.x;
    int tidy = blockIdx.y * blockDim.y + threadIdx.y;
    int tindex = tidx + gridDim.x * BLOCKSIZE * tidy;

    if(tindex < M) {
        int temp = tindex * N;
        for(int i = 0; i < N; i++) {
            c[tindex] += a[temp + i] * b[i];
        }
    }
    __syncthreads();

}


int main() {

    float *a_host, *b_host, *c_host;
    float *a_device, *b_device, *c_device;

    a_host = (float *)malloc(M * N * sizeof(float));
    b_host = (float *)malloc(N * sizeof(float));
    c_host = (float *)malloc(M * sizeof(float));

    
    for(int i = 0; i < M*N; i++) {
        a_host[i] =1024*1024 * float(rand())/RAND_MAX;
    }

    for(int i = 0; i < N; i++) {
        b_host[i] =1024*1024 * float(rand())/RAND_MAX;
    }


    cout<<"INPUT SIZE "<<endl;
    cout<<"Matrix A : "<<M<<" * "<<N<<endl;
    cout<<"Vector B : "<<N<<" * "<<1<<endl;

    cudaMalloc(&a_device, M * N * sizeof(float));
    cudaMalloc(&b_device, N * sizeof(float));
    cudaMalloc(&c_device, M * sizeof(float));


    cudaMemcpy(a_device, a_host, M * N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(b_device, b_host, N * sizeof(float), cudaMemcpyHostToDevice);

    cudaMemset(c_device, 2, M * sizeof(float));

   
	  float gpu_elapsed_time;
	  cudaEvent_t gpu_start, gpu_stop;

	  cudaEventCreate(&gpu_start);
	  cudaEventCreate(&gpu_stop);


	  cudaEventRecord(gpu_start, 0);    

    
    dim3 blockSize(BLOCKSIZE, BLOCKSIZE);
    dim3 gridSize(1, M / (BLOCKSIZE*BLOCKSIZE) + 1);

    auto startGPU = high_resolution_clock::now();
    mat_vec_mul_kernel<<<gridSize, blockSize>>>(a_device, b_device, c_device);

    cudaEventRecord(gpu_stop, 0);
    cudaEventSynchronize(gpu_stop);
    cudaEventElapsedTime(&gpu_elapsed_time, gpu_start, gpu_stop);

    cudaEventDestroy(gpu_start);
    cudaEventDestroy(gpu_stop);

    c_host = (float *)malloc(M * sizeof(float));
    cudaMemcpy(c_host, c_device, M * sizeof(float), cudaMemcpyDeviceToHost);

    cout<<"Answer : \n"<<endl;
    for(int i = 0; i < 5; i++) {
        cout<<c_host[i]<<endl;
    }
    cout<<"\nTime on GPU : "<<gpu_elapsed_time<<" milli seconds\n\n"<<endl;

    // Free allocated Memory
    free(a_host);
    free(b_host);
    free(c_host);
    cudaFree(a_device);
    cudaFree(b_device);
    cudaFree(c_device);

    return 0;

}


tcmalloc: large alloc 4294967296 bytes == 0x55a15d104000 @  0x7fd97524f1e7 0x55a15ba55361 0x7fd974280b97 0x55a15ba5517a
INPUT SIZE 
Matrix A : 1048576 * 1024
Vector B : 1024 * 1
Answer : 

2.84318e+14
2.83432e+14
2.82785e+14
2.85548e+14
2.7928e+14

Time on GPU : 137.821 milli seconds



